In [1]:
import sys
sys.path.append('third_party/Matcha-TTS')

from cosyvoice.cli.cosyvoice import CosyVoice
from cosyvoice.utils.file_utils import load_wav
import torchaudio

cosyvoice = CosyVoice('./pretrained_models/CosyVoice-300M',load_jit=False, load_trt=False)

import torch
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
from cosyvoice.utils.train_utils import (
init_distributed,
init_dataset_and_dataloader,
init_optimizer_and_scheduler,
init_summarywriter, save_model,
wrap_cuda_model, check_modify_and_save_config)
from hyperpyyaml import load_hyperpyyaml

config_file = "./pretrained_models/CosyVoice-300M/cosyvoice.yaml"

override_dict = {k: None for k in ['llm', 'flow', 'hift'] if k != "llm"}

with open(config_file, 'r') as f:
    configs = load_hyperpyyaml(f, overrides=override_dict)

#configs['train_conf'].update(vars(args))

print(configs)

2025-01-13 05:03:33,046 - modelscope - INFO - PyTorch version 2.0.0 Found.
2025-01-13 05:03:33,048 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2025-01-13 05:03:33,084 - modelscope - INFO - Loading done! Current index file version is 1.15.0, with md5 13b7082244438de82b16012a4fd7baad and a total number of 980 components indexed
/data/minju/conda/envs/cosyvoice/lib/python3.10/site-packages/diffusers/models/lora.py:393: FutureWarning: `LoRACompatibleLinear` is deprecated and will be removed in version 1.0.0. Use of `LoRACompatibleLinear` is deprecated. Please switch to PEFT backend by installing PEFT: `pip install peft`.
  deprecate("LoRACompatibleLinear", "1.0.0", deprecation_message)
2025-01-13 05:03:42,263 INFO input frame rate=50
/data/minju/conda/envs/cosyvoice/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:115: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available pr

[2025-01-13 05:04:00,129] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2025-01-13 05:04:00,305 INFO gcc -pthread -B /data/minju/conda/envs/cosyvoice/compiler_compat -Wno-unused-result -Wsign-compare -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /data/minju/conda/envs/cosyvoice/include -fPIC -O2 -isystem /data/minju/conda/envs/cosyvoice/include -fPIC -c /tmp/tmpptz30zo0/test.c -o /tmp/tmpptz30zo0/test.o
2025-01-13 05:04:00,553 INFO gcc -pthread -B /data/minju/conda/envs/cosyvoice/compiler_compat /tmp/tmpptz30zo0/test.o -laio -o /tmp/tmpptz30zo0/a.out
/data/minju/conda/envs/cosyvoice/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  NVIDIA Inference is only supported on Ampere and newer architectures
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.0
 [WARNING]  using untested triton version (2.0.0), only 1.0.0 is known to be compatible
{'sample_rate': 22050, 'text_encoder_input_size': 512, 'llm_input_size': 1024, 'llm_output_size': 1024, 'spk_embed_dim': 192, 'llm': TransformerLM(
  (text_embedding): Embedding(51866, 512)
  (text_encoder): ConformerEncoder(
    (embed): LinearNoSubsampling(
      (out): Sequential(
        (0): Linear(in_features=512, out_features=1024, bia

In [18]:
## 임베딩 파일 경로 변경
print(cosyvoice.list_available_spks())
print(len(cosyvoice.list_available_spks()))

emb_path = "/data/minju/TTS/CosyVoice/embedding_models/son-ploonet/spk2info.pt"
# emb_path = "/data/minju/TTS/CosyVoice/embedding_models/slx/spk2info.pt"
# emb_path = "/data/minju/TTS/CosyVoice/embedding_models/semi/spk2info.pt"
emb_path = "/data/minju/TTS/CosyVoice/embedding_models/senti-slx/spk2info.pt"
emb_path = "/data/minju/TTS/CosyVoice/embedding_models/cardergarden/spk2info.pt"
emb_type = emb_path.split("/")[-2]
spk2info = torch.load(emb_path, map_location = "cuda")
cosyvoice.frontend.spk2info = spk2info

print(cosyvoice.list_available_spks())
print(len(cosyvoice.list_available_spks()))

['tony', 'elley', 'roy', 'eden', 'rudy', 'noa', 'ruby', 'lye', 'jenny', 'secretary_ian', 'secretary_roun', 'secretary_joy', 'bitna_bright', 'bitna_default', 'hayun_bright_1.05', 'hayun_default_1.05', 'kyeon_bright', 'kyeon_default', 'sanggyun_default', 'sehyeok_bright', 'sehyeok_default', 'wansun_default_1.05', 'yongmin_default_1.05', 'yuna_bright', 'yuna_default_1.05', 'boram_bright', 'tony_default_1.05', 'boram_default', 'kyungil_default_1.05', 'slx_speakers_KAIST_males_2', 'slx_speakers_del_JIHYANG', 'slx_speakers_SEHA_fastpitch', 'donghee_default', 'donghee_bright', 'F-A3-D', 'F-H3-D', 'F-NY-D', 'F-S3-D', 'F-A3-A', 'F-H3-A', 'F-NY-A', 'F-S3-A', 'F-A2-A', 'F-H2-A', 'F-S2-A', 'F-A2-E', 'F-H2-E', 'F-NY-E', 'F-S2-E', 'F-A1-C', 'F-H1-C', 'F-NY-C', 'F-S1-C', 'F-NX-A', 'F-A3-B', 'F-H3-B', 'F-NY-B', 'F-S3-B', 'F-A1-Q', 'F-H1-Q', 'F-NX-Q', 'F-S1-Q', 'F-A2-Q', 'F-H2-Q', 'F-NY-Q', 'F-S2-Q', 'N-NX-D', 'D-A2-C', 'D-H2-C', 'D-NX-C', 'D-S2-C', 'D-A2-F', 'D-H2-F', 'D-NX-F', 'D-S2-F', 'D-A1-D', 'D-

In [21]:
from IPython.display import Audio
import os

# 모델 변경
# sft usage
# change stream=True for chunk stream inference
epoch = 20
flag = True

if flag:
    model = configs["llm"]
    model.load_state_dict(torch.load(f"/data/minju/TTS/CosyVoice/examples/libritts/cosyvoice/exp/250111-son-ploonet/llm/torch_ddp/epoch_{epoch}_whole.pt", map_location='cuda'), strict=False)
    cosyvoice.model.llm = model.to("cuda")


In [24]:
speaker = "cardergarden"
text = "오늘부터 사나흘에 걸쳐 충남과 호남 지역 제주도 등에 많은 눈이 내리겠습니다. 이 시각 대설주의보가 내려진 전북 고창의 모습을 보면 거리에 눈이 쌓인 가운데 굵은 눈이 펑펑 쏟아지고 있습니다. 서해안에서 강한 눈구름대가 유입되면서 서쪽 지역을 중심으로 눈이 내리고 있습니다."
# text = "끊임없이 올라가기만 하고 재미없어요. 내려오기도 해야 재밌죠. 오르락 내리락이있으니까 재미라는게 생기는거죠."
# text = "그룹 블랙핑크 로제와 팝스타 브루노 마스가 함께 부른 아파트가 빌보드 차트에서 전주보다 이십 구 계단 상승하며 오위에 올랐습니다."
output_path = "output-emb/250113"
os.makedirs(output_path, exist_ok=True)
save_path = f"{output_path}/sft_{emb_type}_{speaker}_{flag}_{text[:3]}"
for i, j in enumerate(cosyvoice.inference_sft(text, speaker, stream=False)):
    torchaudio.save(f"{save_path}_{i}.wav", j['tts_speech'], cosyvoice.sample_rate)
    #Audio(f"{save_path}_{i}.wav")

2025-01-13 06:20:15,509 INFO synthesis text 오늘부터 사나흘에 걸쳐 충남과 호남 지역 제주도 등에 많은 눈이 내리겠습니다。이 시각 대설주의보가 내려진 전북 고창의 모습을 보면 거리에 눈이 쌓인 가운데 굵은 눈이 펑펑 쏟아지고 있습니다。


text dtype: torch.int32
text_len dtype: torch.int32
prompt_text dtype: torch.int32
prompt_text_len dtype: torch.int32
prompt_speech_token dtype: torch.int32
prompt_speech_token_len dtype: torch.int32
embedding dtype: torch.float32
embedding dtype after normalize: torch.float32
embedding dtype after affine layer: torch.float32


2025-01-13 06:20:39,423 INFO yield speech len 14.675011337868481, rtf 1.629567438850026
2025-01-13 06:20:39,436 INFO synthesis text 서해안에서 강한 눈구름대가 유입되면서 서쪽 지역을 중심으로 눈이 내리고 있습니다。


text dtype: torch.int32
text_len dtype: torch.int32
prompt_text dtype: torch.int32
prompt_text_len dtype: torch.int32
prompt_speech_token dtype: torch.int32
prompt_speech_token_len dtype: torch.int32
embedding dtype: torch.float32
embedding dtype after normalize: torch.float32
embedding dtype after affine layer: torch.float32


2025-01-13 06:20:50,723 INFO yield speech len 6.048798185941043, rtf 1.8659880544790095
100%|██████████| 2/2 [00:35<00:00, 17.61s/it]
